In [1]:
from model4LHzc import *
from data_1ch import *

vid="v007_5" #version id should match the file number 
# The final number can be [1:5] and denote the cross-validation order
# Ran DellWS with GeForce RTX3060 GPU

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='wrap')

# SETTINGS 1 ***
batch_size=4
learning_rate=1e-3

train_gen = trainGenerator(batch_size,'fgt_seg07/train0'+vid[-1],'fgt_seg','fgt_seg_labels',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'fgt_seg07/valid0'+vid[-1],'fgt_seg','fgt_seg_labels',data_gen_args,save_to_dir = None)

# 48 images are used for training, 12 images for validating and 6 images for testing
train_steps = 48//batch_size
valid_steps = 12//batch_size

# SETTINGS 2 ***
loss=dice_loss
steps_per_epoch=3*train_steps
num_epochs=100     
dropout_rate=0.5
num_top_filter=4
#input_size = (960,480,1)

model = unet(learning_rate, loss, num_top_filter, dropout_rate)

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef, iou]
model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files/unet_fgt_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=7, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps, 
                    epochs=num_epochs, callbacks=callbacks)


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 48 images belonging to 1 classes.
Found 48 images belonging to 1 classes.
Epoch 1/100
36/36 [==============================] - ETA: 0s - loss: 0.6932 - acc: 0.3826 - dice_coef: 0.3068 - iou: 0.1817Found 12 images belonging to 1 classes.
Found 12 images belonging to 1 classes.
36/36 [==============================] - 43s 817ms/step - loss: 0.6932 - acc: 0.3826 - dice_coef: 0.3068 - iou: 0.1817 - val_loss: 0.7618 - val_acc: 0.1766 - val_dice_coef: 0.2382 - val_iou: 0.1352

Epoch 00001: saving model to files\unet_fgt_seg_v007_5.hdf5
Epoch 2/100
36/36 [==============================] - 30s 839ms/step - loss: 0.6159 - acc: 0.7397 - dice_coef: 0.3841 - iou: 0.2380 - val_loss: 0.7772 - val_acc: 0.2268 - val_dice_coef: 0.2228 - val_iou: 0.1256

Epoch 00002: saving model to files\unet_fgt_seg_v007_5.hdf5
Epoch 3/100
36/36 [==============================] - 30s 847ms/step - loss: 0.5617 - acc: 0.7715 - dice_coef: 0.4383 - iou: 0.2810 - val_loss: 0.6600 - val_acc: 0.4317 - val_dice_coef: 0.


Epoch 00029: saving model to files\unet_fgt_seg_v007_5.hdf5
Epoch 30/100
36/36 [==============================] - 25s 700ms/step - loss: 0.1507 - acc: 0.8724 - dice_coef: 0.8493 - iou: 0.7395 - val_loss: 0.1799 - val_acc: 0.8414 - val_dice_coef: 0.8201 - val_iou: 0.6960

Epoch 00030: saving model to files\unet_fgt_seg_v007_5.hdf5
Epoch 31/100
36/36 [==============================] - 25s 698ms/step - loss: 0.1573 - acc: 0.8655 - dice_coef: 0.8427 - iou: 0.7296 - val_loss: 0.2630 - val_acc: 0.7554 - val_dice_coef: 0.7370 - val_iou: 0.6072

Epoch 00031: saving model to files\unet_fgt_seg_v007_5.hdf5
Epoch 32/100
36/36 [==============================] - 25s 698ms/step - loss: 0.1509 - acc: 0.8721 - dice_coef: 0.8491 - iou: 0.7397 - val_loss: 0.2181 - val_acc: 0.8016 - val_dice_coef: 0.7819 - val_iou: 0.6498

Epoch 00032: saving model to files\unet_fgt_seg_v007_5.hdf5
Epoch 33/100
36/36 [==============================] - 24s 696ms/step - loss: 0.1545 - acc: 0.8681 - dice_coef: 0.8455 - iou

### validate your model and save predicted results

In [3]:
validGene = testGenerator("fgt_seg07/valid0"+vid[-1]+"/pred")
model.load_weights("files/unet_fgt_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,12,verbose=1)
saveResult("fgt_seg07/valid0"+vid[-1]+"/pred",results,vid[:4])

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


12/12 [==============================] - 2s 69ms/step


In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 960, 480, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 960, 480, 4)  40          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 960, 480, 4)  16          conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 960, 480, 4)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [5]:
from IPython.display import Audio
sound_file = 'Alarm08.wav'
Audio(sound_file, autoplay=True)